# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
output_data_file = "../output_data/cities.csv"
weather_df = weather_df = pd.read_csv(output_data_file)
weather_df

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
## Configure gmaps
gmaps.configure(api_key=g_key)
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Lng"]]

# Store Humidity in hmuidty
humidity = weather_df["Humidity"]

# Plot Heatmap
figure_layout = {
    'width': '1200px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Create ideal weather DataFrame
ideal_weather_df = weather_df.loc[(weather_df["Cloudiness"] == 0) & 
                                  (weather_df["Wind Speed"] < 10) & 
                                  (weather_df["Max Temp"] > 70) &
                                  (weather_df["Max Temp"] < 80)]

ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df                               


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
13,13,satrikh,0,IN,1586122020,50,26.87,81.20,75.20,3.36
178,178,ormara,0,PK,1586122034,59,25.21,64.64,78.46,4.92
207,207,sombrio,0,BR,1586122037,75,-29.11,-49.62,71.20,9.78
254,254,veraval,0,IN,1586122040,62,20.90,70.37,77.18,9.10
263,263,bela,0,IN,1586122041,27,25.93,81.98,77.11,5.48
417,417,hualmay,0,PE,1586122053,63,-11.10,-77.61,77.65,8.66
423,423,beitbridge,0,ZW,1586122054,37,-22.22,30.00,71.87,3.24
452,452,mopipi,0,BW,1586122056,28,-21.18,24.88,74.52,0.60
454,454,badarpur,0,IN,1586122056,48,24.90,92.60,70.36,2.15
473,473,silvassa,0,IN,1586122058,63,20.27,73.02,74.82,2.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = ideal_weather_df.copy()
hotel_df["Hotel Name"] = ""
hotel_df.reset_index(drop=True,inplace=True)
hotel_df

target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# "rankby": "distance"
# set up a parameters dictionary
params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each city
for index,row in hotel_df.iterrows():
    params['location'] = (f"{row['Lat']},{row['Lng']}")

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
   
    try:
        # get first hotel name
        hotel_name = response["results"][0]["name"]
    except IndexError:
        hotel_name=""

    # add hotel name to data frame row
    hotel_df.loc[index, "Hotel Name"] = hotel_name

hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
hotel_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,13,satrikh,0,IN,1586122020,50,26.87,81.20,75.20,3.36,OYO 47218 Surja Guest House And Marriage Lawn
1,178,ormara,0,PK,1586122034,59,25.21,64.64,78.46,4.92,Zero Point Ormara
2,207,sombrio,0,BR,1586122037,75,-29.11,-49.62,71.20,9.78,Pousada Magia da Sereia
3,254,veraval,0,IN,1586122040,62,20.90,70.37,77.18,9.10,Hotel The Grand Daksh Somnath
4,263,bela,0,IN,1586122041,27,25.93,81.98,77.11,5.48,SPOT ON 47288 Shree Ram Inn
5,417,hualmay,0,PE,1586122053,63,-11.10,-77.61,77.65,8.66,La Posada De Santa Maria
6,423,beitbridge,0,ZW,1586122054,37,-22.22,30.00,71.87,3.24,"Elliot Hotel&Conference Centre,Beit Bridge"
9,473,silvassa,0,IN,1586122058,63,20.27,73.02,74.82,2.21,Treat Resort
10,534,sohag,0,EG,1586122063,31,26.56,31.69,75.20,4.70,El Safa Hotel
11,562,quelimane,0,MZ,1586122065,83,-17.88,36.89,77.00,4.70,Hotel Elite


In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:

markers = gmaps.marker_layer(locations,info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…